In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import os, glob

try:
    import cPickle as pickle
except:
    import pickle

from IPython.display import display, display_html

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)
#with pd.option_context('display.float_format', '{:}'.format):

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
root_path = '../'

data_path = root_path + 'data/'

---
## 0. 데이터 확인, 파악

In [6]:
glob.glob(data_path + '*')

['../data/train_labels.csv',
 '../data/test.csv',
 '../data/specs.csv',
 '../data/train.csv',
 '../data/sample_submission.csv']

In [11]:
train = pd.read_csv(data_path + 'train.csv')
train_labels = pd.read_csv(data_path + 'train_labels.csv')
specs = pd.read_csv(data_path + 'specs.csv')
test = pd.read_csv(data_path + 'test.csv')
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')

5개의 데이터 셋
* specs : 이벤트 정의
  * ex) event_id : '1325467d'
  * ex) info : 'This event occurs when the player clicks on something that isn’t covered elsewhere. It can be useful in determining if there are attractive distractions (things the player think should do something, but don’t) in the game, or diagnosing players who are having mechanical difficulties (near misses).'
  * ex) args : '[{"name":"game_time","type":"int","info":"millisecond count since start of game"},{"name":"coordinates","type":"object","info":"the game screen coordinates of the mouse click"},{"name":"event_count","type":"int","info":"session event counter"},{"name":"event_code","type":"int","info":"event class identifier"}]'


* train / test : 발생한 이벤트에 대한 데이터
  * installation_id : 사용자를 구분짓는 key로, test셋에 포함된 installation_id별 마지막 timestamp를 가진 세션의 accuracy_group을 예측하는 것이 목표.
* train_labels : 학습셋의 라벨 정보에 대한 것으로, game_session, installation_id와 결합하여 accuracy_group을 라벨링한다
  * 

In [38]:
len(train_labels.groupby(['game_session', 'installation_id']).size())

17690

In [39]:
len(train.groupby(['game_session', 'installation_id']).size())

303319

In [40]:
len(test.groupby(['game_session', 'installation_id']).size())

28445

In [41]:
len(test.groupby(['installation_id']).size())

1000

In [42]:
train.title.value_counts()

Chow Time                        1150974
Sandcastle Builder (Activity)    1034332
Scrub-A-Dub                      1016837
Bottle Filler (Activity)         1004068
Fireworks (Activity)              535030
                                  ...   
12 Monkeys                          4124
Honey Cake                          3882
Rulers                              3372
Heavy, Heavier, Heaviest            3351
Treasure Map                        2852
Name: title, Length: 44, dtype: int64

In [43]:
test.title.value_counts()

Bottle Filler (Activity)         112223
Scrub-A-Dub                       96619
Chow Time                         93142
Sandcastle Builder (Activity)     82132
Fireworks (Activity)              61032
                                  ...  
12 Monkeys                          430
Costume Box                         422
Rulers                              320
Heavy, Heavier, Heaviest            281
Treasure Map                        223
Name: title, Length: 44, dtype: int64

In [12]:
print("train.csv : ", train.shape)
display(train.head())
print("test.csv : ", test.shape)
display(test.head())

train.csv :  (11341042, 11)


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


test.csv :  (1156414, 11)


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [13]:
print("train_labels.csv : ", train_labels.shape)
display(train_labels.head())

train_labels.csv :  (17690, 7)


,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [37]:
train_labels.groupby(['installation_id', 'title']).size()

installation_id  title                       
0006a69f         Bird Measurer (Assessment)      2
                 Mushroom Sorter (Assessment)    3
0006c192         Bird Measurer (Assessment)      1
                 Cauldron Filler (Assessment)    1
                 Mushroom Sorter (Assessment)    1
                                                ..
ffc90c32         Chest Sorter (Assessment)       1
                 Mushroom Sorter (Assessment)    2
ffd2871d         Cauldron Filler (Assessment)    1
ffeb0b1b         Cauldron Filler (Assessment)    2
                 Mushroom Sorter (Assessment)    1
Length: 9386, dtype: int64

In [17]:
pd.DataFrame(train_labels.installation_id.value_counts())

,installation_id
08987c08,156
f1c21eda,129
9f399a3a,78
8e44d2de,72
96979008,64
...,...
45e5c88e,1
2f607e24,1
7f8d0e15,1
0f2e23b4,1


In [22]:
train_labels[train_labels.installation_id == '96979008'].game_session.value_counts()

f898d5cf892faeeb    1
50793c92b1af4f47    1
230526cc976f3e00    1
be54f095a20520cf    1
c769f8c87adbde40    1
                   ..
884ba46ed9aea029    1
2f57f82f197fc455    1
7688d1866925a30b    1
16bb12ece4d205cb    1
aa534b70fb651c89    1
Name: game_session, Length: 64, dtype: int64

In [25]:
train[train.installation_id == '96979008'].game_session.value_counts()

bd5dfd084933f81e    992
6d7a8a3dec35e867    762
8db7acd6919331f6    682
13e3c60e964c5abf    650
79132d17fb683579    645
                   ... 
6dad27a6fb60b9c8      1
f2a180afef13fa5e      1
9e590a912ad8145e      1
6bfd1476b00f3cf5      1
01343563b601c7bc      1
Name: game_session, Length: 384, dtype: int64

In [14]:
print("sample_submission.csv : ", sample_submission.shape)
display(sample_submission.head())

sample_submission.csv :  (1000, 2)


,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


In [15]:
print("specs.csv : ", specs.shape)
display(specs.head())

specs.csv :  (386, 3)


,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
